# Compare the FY23 VMT-Mix with HPMS VMT-Mix

Created by: Apoorb  
Created on: July 20, 2023

Processing steps:

1. Percent of emissions from Running
2. Refactor running VMT to different vehicles
3. Recompute emissions
4. See the impact


In [11]:
from pathlib import Path
import pandas as pd
import geopandas as gpd
import numpy as np


Subset of pollutants that are important for Texas.


In [12]:
POLS = [
    "Carbon Monoxide (CO)",
    "Oxides of Nitrogen (NOx)",
    "Volatile Organic Compounds",
    "CO2 Equivalent",
    "Primary Exhaust PM10  - Total",
    "Primary PM10 - Brakewear Particulate",
    "Primary PM10 - Tirewear Particulate",
    "Primary Exhaust PM2.5 - Total",
    "Primary PM2.5 - Brakewear Particulate",
    "Primary PM2.5 - Tirewear Particulate",
]


## Load Data

1. Read
   - _trends_2020_sutft_ei.csv and trends_2020_sutft_act.csv_: trends emission and activity data,
   - _fy23_fin_vmtmix_13_21_032023_: FY23 (2013 to 2021 Oct) VMT-Mix,
   - _PivotChartForVMTmixComparison.xlsx_: HyeMin's HPMS VMT-Mix,
   - _Rate_Categories_: Rate categories to map emissions to activity,
   - _Texas_County_Boundaries & TxDOT_Districts_: Texas counties and districts shapefile to connect the districts with counties.
2. Fix data types.
3. Filter VMT-Mix to only keep day daily VMT-Mix for MOVES road types.


In [13]:
TOD_FILT = "day"
TOD_FILT = "AM"
OUT_EMIS_FI = f"raw_trends_fy23_hpms_emis_{TOD_FILT}_mix_11Aug2023"


In [14]:
pa_data = Path(r"C:\Users\a-bibeka\PycharmProjects\FY23_VMT_Mix\data")
pa_emis = pa_data.joinpath("trends_2020_sutft_ei.csv")
pa_act = pa_data.joinpath("trends_2020_sutft_act.csv")
pa_vmix_dp = pa_data.joinpath("dennis_vmtmix_prc.csv")
pa_vmix_fy22 = pa_data.joinpath("fy22_fin_vmtmix_13_19_042022.csv")
pa_vmix_fy23 = pa_data.joinpath("fy23_fin_vmtmix_13_21_032023.csv")
pa_vmix_hpms = pa_data.joinpath("PivotChartForVMTmixComparison.xlsx")
pa_rate_map = pa_data.joinpath("Rate_Categories.csv")
pa_county_shp = pa_data.joinpath("Texas_County_Boundaries", "County.shp")
pa_district_shp = pa_data.joinpath("TxDOT_Districts", "TxDOT_Districts.shp")
sutft_act = pd.read_csv(pa_act)
suft_emis = pd.read_csv(pa_emis)
vmtmix_dp = pd.read_csv(pa_vmix_dp)
vmtmix_fy22 = pd.read_csv(pa_vmix_fy22)
vmtmix_fy23 = pd.read_csv(pa_vmix_fy23)
rate_map = pd.read_csv(pa_rate_map)
gdf_county = gpd.read_file(pa_county_shp)
gdf_district = gpd.read_file(pa_district_shp)
df_district = gdf_district.filter(items=["DIST_NM", "DIST_NBR"])
district_map = gdf_county[["TXDOT_DIST", "FIPS_ST_CN", "CNTY_NM"]].astype(
    {"TXDOT_DIST": int, "FIPS_ST_CN": int, "CNTY_NM": str}
)
x1 = pd.ExcelFile(pa_vmix_hpms)
vmtmix_hpms = x1.parse("DatasetForPivotChart", skiprows=4).loc[
    lambda df: (df.Year == 2020) & (df.DayType == "Wkd") & (df.TimeType == "day")
]

vmtmix_dp = vmtmix_dp.rename(columns={"dow": "dowagg", "year": "yearID"})
map_rdty = {"r_ra": 2, "r_ura": 3, "u_ra": 4, "u_ura": 5}
vmtmix_dp["mvs_rdtype"] = vmtmix_dp["mvs_rdtype_nm"].map(map_rdty)
vmtmix_filt_dp = vmtmix_dp.loc[
    lambda df: (df.dowagg == "Wkd")
    & (df.yearID == 2020)
    & (df.tod == TOD_FILT)
    & (df.mvs_rdtype_nm != "ALL")
].rename(
    columns={
        "sourceTypeID": "source_type_id",
        "fuelTypeID": "fuel_type_id",
        "vmt_mix": "vmt_mix_dp",
    }
)

vmtmix_filt_fy22 = vmtmix_fy22.loc[
    lambda df: (df.dowagg == "Wkd")
    & (df.yearID == 2020)
    & (df.tod == TOD_FILT)
    & (df.mvs_rdtype_nm != "ALL")
].rename(
    columns={
        "sourceTypeID": "source_type_id",
        "fuelTypeID": "fuel_type_id",
        "vmt_mix": "vmt_mix_fy22",
    }
)

vmtmix_filt_fy23 = vmtmix_fy23.loc[
    lambda df: (df.dowagg == "Wkd")
    & (df.yearID == 2020)
    & (df.tod == TOD_FILT)
    & (df.mvs_rdtype_nm != "ALL")
].rename(
    columns={
        "sourceTypeID": "source_type_id",
        "fuelTypeID": "fuel_type_id",
        "vmt_mix": "vmt_mix_fy23",
    }
)


In [15]:
rename_sutft_dat = {
    "Period": "period",
    "Year": "year_id",
    "FIPS": "fips",
    "County": "cnty_nm",
    "Road Type ID": "mvs_rdtype",
    "Road Type": "rdtype",
    "Source Type ID": "source_type_id",
    "Source Type": "sut",
    "Fuel Type ID": "fuel_type_id",
    "Fuel Type": "ft",
    "Source Type_Fuel Type Label": "sutft_lab",
    "sort_order_pol": "sort_order_pol",
    "MOVES3 Pollutant ID": "pollutant_id",
    "Short Name MOVES3": "short_name_mvs3",
    "EIS Pollutant Code": "eis_polcd",
    "NEI Pollutant Name": "short_name_nei17",
    "excluded_pollutant_from_xml": "excluded_pollutant_from_xml",
    "Process ID": "process_id",
    "Process Label": "prc_lab",
    "Process Name": "prc_nm",
    "MOVES3 SCC": "mvs_scc",
    "NEI SCC": "nei_scc",
    "txled_fac": "txled_fac",
    "Emission (Tons)": "emis_short_tons",
    "Activity Type ID": "actty_id",
    "Activity Type Abb": "acttyabb",
    "Activity Type": "acttylab",
    "activity": "act",
    "Units": "act_unit",
}


## Transform Data


### Get District County Map

Use _Texas_County_Boundaries & TxDOT_Districts_ shapefiles to connect Texas counties and districts.


In [16]:
fips_254 = list(district_map.FIPS_ST_CN.unique())
analysis_fips = fips_254
district_map_anly = district_map.loc[
    district_map.FIPS_ST_CN.isin(analysis_fips)
].rename(
    columns={"FIPS_ST_CN": "fips", "CNTY_NM": "cnty_nm", "TXDOT_DIST": "txdot_dist"}
)

df_district = df_district.rename(
    columns={"DIST_NM": "district", "DIST_NBR": "txdot_dist"}
)
district_map_anly = df_district.merge(district_map_anly, on="txdot_dist")
district_map_anly.loc[lambda df: df.district == "Fort Worth"]


,district,txdot_dist,fips,cnty_nm
113,Fort Worth,2,48221,Hood
114,Fort Worth,2,48251,Johnson
115,Fort Worth,2,48363,Palo Pinto
116,Fort Worth,2,48367,Parker
117,Fort Worth,2,48425,Somervell
118,Fort Worth,2,48143,Erath
119,Fort Worth,2,48497,Wise
120,Fort Worth,2,48439,Tarrant
121,Fort Worth,2,48237,Jack


### Process HPMS VMT-Mix


In [17]:
# vmtmix_hpms.groupby(["Year", "RoadType", "DayType", "TimeType", "District"]).agg(
#     vmt_hpms=("Fraction-HPMS", "sum")
# ).describe()


In [18]:
# vmtmix_hpms.groupby(["Year", "RoadType", "DayType", "TimeType", "District"]).agg(
#     vmt_fy23=("Fraction", "sum")
# ).describe()


In [19]:
# Map the HPMS categories in HyeMin's table to a code.
map_vcat = {
    "Motorcycles": 10,
    "Passenger Cars": 251,
    "Light truck": 252,
    "Bus": 40,
    "ST": 50,
    "CT": 60,
}

# Map MOVES SUT to HyeMin's HPMS categories
map_sut = {
    11: 10,
    21: 251,
    31: 252,
    32: 252,
    41: 40,
    42: 40,
    43: 40,
    51: 50,
    52: 50,
    53: 50,
    54: 50,
    61: 60,
    62: 60,
}

# Mapping to connect HPMS road types to MOVES road types
map_hpms_idx_lab = {
    1: "Urban Interstate",
    2: "Rural Interstate",
    3: "Urban Other Arterials",
    4: "Urban Others",
    5: "Rural Other Arterials",
    6: "Rural Others",
}
map_hpms_mvs = {1: "u_ra", 2: "r_ra", 3: "u_ura", 4: "u_ura", 5: "r_ura", 6: "r_ura"}
mvs_lab_idx = {"r_ra": 2, "r_ura": 3, "u_ra": 4, "u_ura": 5}
map_hpms_mvs_idx = {key: mvs_lab_idx[val] for key, val in map_hpms_mvs.items()}


In [20]:
# Code the HPMS categories in HyeMin's table.
vmtmix_hpms = vmtmix_hpms.rename(columns={"Vehicle type": "hpmsvehtype"})
vmtmix_hpms.hpmsvehtype.unique()

vmtmix_hpms["hpms_idx"] = vmtmix_hpms.hpmsvehtype.map(map_vcat)

# Map MOVES SUT to HyeMin's HPMS categories
vmtmix_filt_fy23["hpms_idx"] = vmtmix_filt_fy23.source_type_id.map(map_sut)

# Rename columns to snake_case
vmtmix_hpms = vmtmix_hpms.rename(
    columns={
        "Vehicle type": "hpmsvehtype",
        "Year": "yearID",
        "RoadType": "hpms_rdtype",
        "DayType": "dowagg",
        "TimeType": "tod",
        "District": "district",
        "Fraction-HPMS": "vmtmix_hpms_agg",
        "Fraction": "vmtmix_axb_agg",
        "Difference (TTI-HPMS)": "vmtmix_axb_hpms_agg_diff",
        "% of Difference": "vmtmix_axb_hpms_agg_perdiff",
    }
)

# Split the HPMS VMT-Mix to MOVES SUT+FT VMT-Mix, by using the internal distribution
# of FY23 VMT-Mix
vmtmix_filt_fy23_1 = (
    vmtmix_filt_fy23.filter(
        items=[
            "district",
            "mvs_rdtype",
            "source_type_id",
            "fuel_type_id",
            "hpms_idx",
            "vmt_mix_fy23",
        ]
    )
    .assign(
        vmt_mix_agg=lambda df: df.groupby(
            ["district", "mvs_rdtype", "hpms_idx"]
        ).vmt_mix_fy23.transform(sum),
        frac_sut_in_hpms=lambda df: df.vmt_mix_fy23 / df.vmt_mix_agg,
    )
    .drop(columns="vmt_mix_fy23")
)

# Connect HPMS road types to MOVES road types
# Then, split the HPMS VMT-Mix to MOVES SUT+FT VMT-Mix, by using the internal distribution of FY23 VMT-Mix
vmtmix_hpms["mvs_rdtype"] = vmtmix_hpms.hpms_rdtype.map(map_hpms_mvs_idx).astype(str)
vmtmix_hpms["hpms_rdtype_nm"] = vmtmix_hpms.hpms_rdtype.map(map_hpms_idx_lab)
vmtmix_hpms_1 = vmtmix_hpms.merge(
    vmtmix_filt_fy23_1, on=["district", "mvs_rdtype", "hpms_idx"]
)
vmtmix_hpms_1["vmt_mix_hpms_sut"] = (
    vmtmix_hpms_1.vmtmix_hpms_agg * vmtmix_hpms_1.frac_sut_in_hpms
)

# Filter to keep relevant columns
vmtmix_hpms_2 = vmtmix_hpms_1.filter(
    items=[
        "district",
        "yearID",
        "dowagg",
        "tod",
        "hpms_rdtype",
        "hpms_rdtype_nm",
        "mvs_rdtype",
        "source_type_id",
        "fuel_type_id",
        "vmt_mix_hpms_sut",
    ]
)

# Assert all districts are present before merging with district-county map.
assert (
    set(district_map_anly.district.unique()).symmetric_difference(
        set(vmtmix_hpms_2.district.unique())
    )
    == set(),
    "Did not find all districts!",
)
vmtmix_hpms_cnty = vmtmix_hpms_2.merge(district_map_anly, on="district")

# Filter to keep relevant columns
vmtmix_hpms_cnty_1 = vmtmix_hpms_cnty.filter(
    items=[
        "hpms_rdtype",
        "hpms_rdtype_nm",
        "fips",
        "cnty_nm",
        "district",
        "mvs_rdtype",
        "source_type_id",
        "fuel_type_id",
        "vmt_mix_hpms_sut",
    ]
)

# Filter to keep 1: "Urban Interstate", 2: "Rural Interstate",
# 3: "Urban Other Arterials", and 5: "Rural Other Arterials",
vmtmix_hpms_cnty_2 = vmtmix_hpms_cnty_1.loc[
    lambda df: df.hpms_rdtype.isin([1, 2, 3, 5])
]


<>:76: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:76: SyntaxWarning: assertion is always true, perhaps remove parentheses?
C:\Users\a-bibeka\AppData\Local\Temp\ipykernel_18672\545758204.py:76: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (


In [21]:
vmtmix_hpms_cnty_2.to_csv(
    pa_data.joinpath("vmtmix_hpms_by_cnty_10Aug2023.csv"), index=False
)


In [22]:
# Test HyeMin's tables VMT-Mix matches the VMT-Mix from AxB's csv file.
assert np.allclose(vmtmix_hpms_1.vmtmix_axb_agg, vmtmix_hpms_1.vmt_mix_agg, atol=1e6)


In [23]:
# QC the above HPMS VMT-Mix by pivoting
vmtmix_hpms_pvttab = vmtmix_hpms_1.pivot_table(
    index=["hpms_idx", "hpms_rdtype", "hpmsvehtype"],
    columns=["source_type_id", "fuel_type_id"],
    values=["vmtmix_hpms_agg", "frac_sut_in_hpms"],
)
vmtmix_hpms_pvttab.to_csv(r"C:\Users\a-bibeka\Downloads\vmt_hpms_pivot_table.csv")


### Process DP, FY22, FY23 VMT-Mix


In [24]:
# Expand VMT-Mix from 25 districts to all 254 counties
vmtmix_filt_dp_anly = vmtmix_filt_dp.merge(
    district_map_anly, on=["district"], how="outer"
)
# Filter to keep relevant columns
vmtmix_filt_dp_anly_1 = vmtmix_filt_dp_anly.filter(
    items=[
        "fips",
        "cnty_nm",
        "district",
        "mvs_rdtype",
        "source_type_id",
        "fuel_type_id",
        "vmt_mix_dp",
    ]
)


In [25]:
# Expand VMT-Mix from 25 districts to all 254 counties
vmtmix_filt_fy22_anly = vmtmix_filt_fy22.merge(
    district_map_anly, on=["txdot_dist", "district"], how="outer"
)
# Filter to keep relevant columns
vmtmix_filt_fy22_anly_1 = vmtmix_filt_fy22_anly.filter(
    items=[
        "fips",
        "cnty_nm",
        "district",
        "mvs_rdtype",
        "source_type_id",
        "fuel_type_id",
        "vmt_mix_fy22",
    ]
)


In [26]:
# Expand VMT-Mix from 25 districts to all 254 counties
vmtmix_filt_fy23_anly = vmtmix_filt_fy23.merge(
    district_map_anly, on=["txdot_dist", "district"], how="outer"
)
# Filter to keep relevant columns
vmtmix_filt_fy23_anly_1 = vmtmix_filt_fy23_anly.filter(
    items=[
        "fips",
        "cnty_nm",
        "district",
        "mvs_rdtype",
        "source_type_id",
        "fuel_type_id",
        "vmt_mix_fy23",
    ]
)


### Get Pollutants of Interest


In [27]:
suft_emis = suft_emis.rename(columns=rename_sutft_dat)
suft_emis1 = suft_emis.loc[lambda df: df.period == "annual"]


In [28]:
rate_map_filt = rate_map.loc[lambda df: df.pollutantName.isin(POLS)]
pol_filt = (
    rate_map_filt[["pollutantID", "pollutantName"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
pol_emis = (
    suft_emis[["pollutant_id", "short_name_mvs3", "short_name_nei17"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
pol_filt_1 = pol_filt.merge(pol_emis, left_on="pollutantID", right_on="pollutant_id")
pols_nei17_ls = pol_filt_1.short_name_nei17.unique()


In [29]:
# VOCs have Evap processes which are only applicable to gasoline fuel. It messes up VMT-Mix aggregation.
pols_nei17_ls1 = set(pols_nei17_ls) - {"VOC"}


### Process Trends 2020 Emission and VMT


In [30]:
suft_emis2 = suft_emis1.loc[lambda df: df.short_name_nei17.isin(pols_nei17_ls1)]
suft_emis3 = (
    suft_emis2.groupby(
        [
            "fips",
            "cnty_nm",
            "mvs_rdtype",
            "rdtype",
            "source_type_id",
            "fuel_type_id",
            "short_name_nei17",
            "process_id",
            "prc_lab",
        ],
        as_index=False,
    )
    .emis_short_tons.sum()
    .rename(columns={"emis_short_tons": "emisSrTon_trend"})
)


In [31]:
sutft_act = sutft_act.rename(columns=rename_sutft_dat)
sutft_act1 = sutft_act.loc[lambda df: df.period == "annual"]


In [32]:
sutft_vmt = sutft_act1.loc[sutft_act.acttyabb == "vmt"]
sutft_vmt = sutft_vmt.rename(columns={"act": "vmt"})
sutft_vmt["vmt_agg"] = sutft_vmt.groupby(
    [
        "cnty_nm",
        "fips",
        "year_id",
        "rdtype",
        "mvs_rdtype",
        "actty_id",
        "acttyabb",
        "acttylab",
    ]
).vmt.transform(sum)

sutft_vmt = sutft_vmt.filter(
    items=[
        "cnty_nm",
        "fips",
        "rdtype",
        "mvs_rdtype",
        "actty_id",
        "acttyabb",
        "acttylab",
        "source_type_id",
        "fuel_type_id",
        "vmt",
        "vmt_agg",
    ]
).rename(columns={"vmt": "vmt_trend", "vmt_agg": "vmt_agg_trend"})
sutft_vmt["vmt_mix_trend"] = sutft_vmt["vmt_trend"] / sutft_vmt["vmt_agg_trend"]


### Merge Emission to VMT


In [33]:
suft_emis_vmt = suft_emis3.merge(
    sutft_vmt,
    on=[
        "cnty_nm",
        "fips",
        "mvs_rdtype",
        "rdtype",
        "source_type_id",
        "fuel_type_id",
    ],
    how="left",
)
suft_emis_vmt_onroad = suft_emis_vmt.loc[
    lambda df: (df.mvs_rdtype != 1) & (df.vmt_trend != 0)
]

suft_emis_vmt_onroad["emisrtcalc_trend"] = (
    suft_emis_vmt_onroad.emisSrTon_trend / suft_emis_vmt_onroad.vmt_trend
)

suft_emis_vmt_onroad_anly = suft_emis_vmt_onroad.loc[
    lambda df: df.fips.isin(analysis_fips)
]


C:\Users\a-bibeka\AppData\Local\Temp\ipykernel_18672\2696073180.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  suft_emis_vmt_onroad["emisrtcalc_trend"] = (


In [34]:
vmtmix_hpms_cnty_2.columns


Index(['hpms_rdtype', 'hpms_rdtype_nm', 'fips', 'cnty_nm', 'district',
       'mvs_rdtype', 'source_type_id', 'fuel_type_id', 'vmt_mix_hpms_sut'],
      dtype='object')

In [35]:
vmtmix_filt_dp_anly_1["fips"] = vmtmix_filt_dp_anly_1["fips"].astype(int)
vmtmix_filt_dp_anly_1["mvs_rdtype"] = vmtmix_filt_dp_anly_1["mvs_rdtype"].astype(int)
vmtmix_filt_fy22_anly_1["fips"] = vmtmix_filt_fy22_anly_1["fips"].astype(int)
vmtmix_filt_fy22_anly_1["mvs_rdtype"] = vmtmix_filt_fy22_anly_1["mvs_rdtype"].astype(
    int
)
vmtmix_filt_fy23_anly_1["fips"] = vmtmix_filt_fy23_anly_1["fips"].astype(int)
vmtmix_filt_fy23_anly_1["mvs_rdtype"] = vmtmix_filt_fy23_anly_1["mvs_rdtype"].astype(
    int
)

vmtmix_hpms_cnty_2["fips"] = vmtmix_hpms_cnty_2["fips"].astype(int)
vmtmix_hpms_cnty_2["mvs_rdtype"] = vmtmix_hpms_cnty_2["mvs_rdtype"].astype(int)
trends_fy23_emis = suft_emis_vmt_onroad_anly.merge(
    vmtmix_filt_fy23_anly_1,
    on=["fips", "mvs_rdtype", "source_type_id", "fuel_type_id"],
    how="left",
    suffixes=["_trend", "_fy23"],
)

trends_fy23_hpms_emis = trends_fy23_emis.merge(
    vmtmix_hpms_cnty_2,
    on=["fips", "mvs_rdtype", "source_type_id", "fuel_type_id"],
    how="left",
    suffixes=["_trend", "_hpms"],
)

trends_fy23_hpms_fy22_dp_emis = trends_fy23_hpms_emis.merge(
    vmtmix_filt_fy22_anly_1,
    on=["fips", "mvs_rdtype", "source_type_id", "fuel_type_id"],
    how="left",
    suffixes=["", "_fy22"],
).merge(
    vmtmix_filt_dp_anly_1,
    on=["fips", "mvs_rdtype", "source_type_id", "fuel_type_id"],
    how="left",
    suffixes=["", "_dp"],
)


C:\Users\a-bibeka\AppData\Local\Temp\ipykernel_18672\2263046932.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vmtmix_hpms_cnty_2["fips"] = vmtmix_hpms_cnty_2["fips"].astype(int)
C:\Users\a-bibeka\AppData\Local\Temp\ipykernel_18672\2263046932.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vmtmix_hpms_cnty_2["mvs_rdtype"] = vmtmix_hpms_cnty_2["mvs_rdtype"].astype(int)


In [36]:
trends_fy23_hpms_fy22_dp_emis.columns


Index(['fips', 'cnty_nm_trend', 'mvs_rdtype', 'rdtype', 'source_type_id',
       'fuel_type_id', 'short_name_nei17', 'process_id', 'prc_lab',
       'emisSrTon_trend', 'actty_id', 'acttyabb', 'acttylab', 'vmt_trend',
       'vmt_agg_trend', 'vmt_mix_trend', 'emisrtcalc_trend', 'cnty_nm_fy23',
       'district_trend', 'vmt_mix_fy23', 'hpms_rdtype', 'hpms_rdtype_nm',
       'cnty_nm', 'district_hpms', 'vmt_mix_hpms_sut', 'cnty_nm_fy22',
       'district', 'vmt_mix_fy22', 'cnty_nm_dp', 'district_dp', 'vmt_mix_dp'],
      dtype='object')

In [42]:
test1 = trends_fy23_hpms_fy22_dp_emis.groupby(
    ["fips", "mvs_rdtype", "short_name_nei17", "prc_lab"], as_index=False
).vmt_mix_fy23.sum()
assert np.allclose(test1.vmt_mix_fy23, 1, atol=0.01), "VMT-Mix doesn't sum up to 1"

test2 = trends_fy23_hpms_fy22_dp_emis.groupby(
    ["fips", "mvs_rdtype", "short_name_nei17", "prc_lab"], as_index=False
).vmt_mix_hpms_sut.sum()
assert np.allclose(test2.vmt_mix_hpms_sut, 1, atol=0.001), "VMT-Mix doesn't sum up to 1"


test3 = trends_fy23_hpms_fy22_dp_emis.groupby(
    ["fips", "mvs_rdtype", "short_name_nei17", "prc_lab"], as_index=False
).vmt_mix_fy22.sum()
assert np.allclose(test3.vmt_mix_fy22, 1, atol=0.01), "VMT-Mix doesn't sum up to 1"


test4 = trends_fy23_hpms_fy22_dp_emis.groupby(
    ["fips", "mvs_rdtype", "short_name_nei17", "prc_lab"], as_index=False
).vmt_mix_dp.sum()
assert np.allclose(test4.vmt_mix_dp, 1, atol=0.001), "VMT-Mix doesn't sum up to 1"


### Compute Emissions for FY23 VMT-Mix


In [43]:
trends_fy23_hpms_fy22_dp_emis[
    "vmt_mix_fy23_norm"
] = trends_fy23_hpms_fy22_dp_emis.vmt_mix_fy23 / trends_fy23_hpms_fy22_dp_emis.groupby(
    ["fips", "mvs_rdtype", "short_name_nei17", "prc_lab"]
).vmt_mix_fy23.transform(
    sum
)

trends_fy23_hpms_fy22_dp_emis[
    "vmt_mix_hpms_sut_norm"
] = trends_fy23_hpms_fy22_dp_emis.vmt_mix_hpms_sut / trends_fy23_hpms_fy22_dp_emis.groupby(
    ["fips", "mvs_rdtype", "short_name_nei17", "prc_lab"]
).vmt_mix_hpms_sut.transform(
    sum
)


trends_fy23_hpms_fy22_dp_emis[
    "vmt_mix_fy22_norm"
] = trends_fy23_hpms_fy22_dp_emis.vmt_mix_fy22 / trends_fy23_hpms_fy22_dp_emis.groupby(
    ["fips", "mvs_rdtype", "short_name_nei17", "prc_lab"]
).vmt_mix_fy22.transform(
    sum
)

trends_fy23_hpms_fy22_dp_emis[
    "vmt_mix_dp_norm"
] = trends_fy23_hpms_fy22_dp_emis.vmt_mix_dp / trends_fy23_hpms_fy22_dp_emis.groupby(
    ["fips", "mvs_rdtype", "short_name_nei17", "prc_lab"]
).vmt_mix_dp.transform(
    sum
)


In [44]:
trends_fy23_hpms_fy22_dp_emis["vmt_fy23"] = (
    trends_fy23_hpms_fy22_dp_emis.vmt_agg_trend
    * trends_fy23_hpms_fy22_dp_emis.vmt_mix_fy23_norm
)
trends_fy23_hpms_fy22_dp_emis["emisSrTon_fy23"] = (
    trends_fy23_hpms_fy22_dp_emis.vmt_fy23
    * trends_fy23_hpms_fy22_dp_emis.emisrtcalc_trend
)

trends_fy23_hpms_fy22_dp_emis["vmt_hpms"] = (
    trends_fy23_hpms_fy22_dp_emis.vmt_agg_trend
    * trends_fy23_hpms_fy22_dp_emis.vmt_mix_hpms_sut_norm
)
trends_fy23_hpms_fy22_dp_emis["emisSrTon_hpms"] = (
    trends_fy23_hpms_fy22_dp_emis.vmt_hpms
    * trends_fy23_hpms_fy22_dp_emis.emisrtcalc_trend
)

trends_fy23_hpms_fy22_dp_emis["vmt_fy22"] = (
    trends_fy23_hpms_fy22_dp_emis.vmt_agg_trend
    * trends_fy23_hpms_fy22_dp_emis.vmt_mix_fy22_norm
)
trends_fy23_hpms_fy22_dp_emis["emisSrTon_fy22"] = (
    trends_fy23_hpms_fy22_dp_emis.vmt_fy22
    * trends_fy23_hpms_fy22_dp_emis.emisrtcalc_trend
)

trends_fy23_hpms_fy22_dp_emis["vmt_dp"] = (
    trends_fy23_hpms_fy22_dp_emis.vmt_agg_trend
    * trends_fy23_hpms_fy22_dp_emis.vmt_mix_dp_norm
)
trends_fy23_hpms_fy22_dp_emis["emisSrTon_dp"] = (
    trends_fy23_hpms_fy22_dp_emis.vmt_dp
    * trends_fy23_hpms_fy22_dp_emis.emisrtcalc_trend
)


In [45]:
trends_fy23_hpms_fy22_dp_emis1 = trends_fy23_hpms_fy22_dp_emis.groupby(
    ["district_trend", "fips", "short_name_nei17"], as_index=False
).agg(
    cnty_nm_trend=("cnty_nm_trend", "first"),
    cnty_nm_fy23=("cnty_nm_fy23", "first"),
    vmt_trend=("vmt_trend", sum),
    vmt_fy23=("vmt_fy23", sum),
    vmt_hpms=("vmt_hpms", sum),
    vmt_fy22=("vmt_fy22", sum),
    vmt_dp=("vmt_dp", sum),
    emisSrTon_trend=("emisSrTon_trend", sum),
    emisSrTon_fy23=("emisSrTon_fy23", sum),
    emisSrTon_hpms=("emisSrTon_hpms", sum),
    emisSrTon_fy22=("emisSrTon_fy22", sum),
    emisSrTon_dp=("emisSrTon_dp", sum),
)


In [46]:
trends_fy23_hpms_fy22_dp_emis.to_csv(pa_data.joinpath(OUT_EMIS_FI), index=False)
